---
title: 'Cohort Analysis for Ironhack Payments'
format: 
    html: 
        toc: true
        page-layout: full
        grid:
            body-width: 1600px
jupyter: python3
---

In [ ]:
# To avoid confusion, we will disable warnings
import warnings
warnings.filterwarnings('ignore')

## Exploratory Data Analysis

> First, let's make sure we have the required modules

In [ ]:
# uncomment the rows bellow to install the modules
# !pip install numpy
# !pip install pandas
# !pip install seaborn
# !pip install matplotlib

:::{.callout-tip}
When using environments, replace !pip with %pip
:::

> Now let's import the modules

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

> And now let's load the CSV files we'll be using

In [ ]:
cash_df = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees_df = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

> Now we can start our exploratory analysis.

:::{.callout-note}
With `pandas` we can use [`.head()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) or [`.tail()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html), [`.info()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [`.shape`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html) and [`.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) to get a quick view of the data.  
We can also use [`.sample(<number of samples>)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) to see some randomly selected rows.
:::

In [ ]:
cash_df.shape

In [ ]:
cash_df.sample(5)

In [ ]:
cash_df.info()

In [ ]:
fees_df.shape

In [ ]:
fees_df.sample(5)

In [ ]:
fees_df.info()

> We already notice that we have some columns with **numbers** (`amount` in `cash_df` and `total_amount` in `fees_df`), **datetime** (`created_at`, `updated_at` etc) and **object** (`status`, `transfer_type` etc).  
> Let's list the `unique()` values for latter.

In [ ]:
print(f"status: {cash_df['status'].unique()}")
print(f"transfer_type: {cash_df['transfer_type'].unique()}")
print(f"recover_status: {cash_df['recovery_status'].unique()}")

:::{.callout-important}
Let's make a note that we have some `nan` (not-a-number) values - we'll get to that in a bit.
:::

In [ ]:
print(f"type: {fees_df['type'].unique()}")
print(f"status: {fees_df['status'].unique()}")
print(f"category: {fees_df['category'].unique()}")
print(f"charge_moment: {fees_df['charge_moment'].unique()}")

## Data Quality Analysis Report

> Now that we've got an idea of the data, let's check if there are any missing (empty or `null`) values in our dataframes.  
> We can use [`.isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) (or its alias - `isnull()`) and the inverse [`notna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.notna.html#pandas.DataFrame.notna) to check if a value is **null**.

In [ ]:
cash_df.isna().sum() # by appending .sum() we get a sum of Null values, per row

> We notice that the `user_id` column has 2103 `nan` values - but we also have a `deleted_account_id` column which contains id's for deleted user accounts.  
Let's check if there are rows where both `user_id` and `deleted_account_id` are empty.

In [ ]:
(cash_df['user_id'].isna() & (cash_df['deleted_account_id'].isna())).sum()

> For the rest of the columns `nan` values are not necessarily a problem, so we won't investigate them further.  
> Now let's look at `fees_df`

In [ ]:
fees_df.isna().sum()

> We notice there are 4 rows that don't have a value for `cash_request_id`

:::{.callout-tip}
The `isna()` function creates a **mask** with `True/False` where the values in the dataframe are `Null`. We can then use this **mask** to select rows from the dataframe
:::

In [ ]:
fees_df[fees_df['cash_request_id'].isna()]

> As the `cash_request_id` is a reference to the `id` from the `cash_df` dataframe, and there are only 4 rows, we can safely remove/ignore these instances.

:::{.callout-tip}
We can use the inverse method `.notna()` and re-assign `fees_df` to the dataframe without `Null` values.
:::

In [ ]:
fees_df = fees_df[fees_df['cash_request_id'].notna()]
fees_df.isna().sum()

## Metrics Analysis

> Before we can actually do our analysis, we need to prepare the dataframes for easier manipulation.  
> For this we'll convert the datetime values to the correct type.

In [ ]:
datetime_columns_cash = ['created_at', 'updated_at', 'moderated_at', 'reimbursement_date',\
                          'cash_request_received_date', 'money_back_date', 'send_at',\
                         'reco_creation', 'reco_last_update']

for column in datetime_columns_cash:
    cash_df[column] = pd.to_datetime(cash_df[column]) # in python3.11.8 we need to add format='mixed'

In [ ]:
cash_df.info()

In [ ]:
datetime_columns_fees = ['created_at', 'updated_at', 'paid_at', 'from_date', 'to_date']
for column in datetime_columns_fees:
    fees_df[column] = pd.to_datetime(fees_df[column])

In [ ]:
fees_df.info()

> Now that our dataframes are cleaned and the values are of the correct type, we can start our analysis.

### 1. Frequency of Service Usage

**Understand how often users from each cohort utilize IronHack Payments' cash advance services over time.**

> To determine the utilisation of the IronHack Payments' service, we first need to group users in cohorts.  
> We will use the `created_at` field to generate the user cohorts, split by months.

In [ ]:
cash_df['cohort'] = cash_df['created_at'].dt.to_period('M')

In [ ]:
cohort_counts = cash_df.groupby('cohort').size()
cohort_counts

> By analysing the cohort sizes, we can deduct that the data is incomplete for the first (2019-11) and last (2020-11) months - to avoid confusion we will hide these months from our plot.

In [ ]:
cohort_counts = cohort_counts.iloc[1:-1]

In [ ]:
plt.figure(figsize=(10, 6))
freq_use = sns.barplot(x=cohort_counts.index.astype(str), y=cohort_counts.values, palette='magma_r')
plt.title('Number of Cash Requests per Cohort')
plt.xlabel('Cohort')
plt.ylabel('Number of Cash Requests')
plt.xticks(rotation=45)
plt.tight_layout()
# we can also show the values directly in the plot
for i in freq_use.containers:
    freq_use.bar_label(i,)

plt.show()

### 2. Incident Rate
**Determine the incident rate, specifically focusing on payment incidents, for each cohort. Identify if there are variations in incident rates among different cohorts.**

> To determine the **Incident Rate** we will look at the `fees_df` dataframe.  
> More specifically, we will consider an **incident** all the rows where `category` is *rejected_direct_debit* or *month_delay_on_payment*.  
> To actually determine the **rate** we need to calculate the percentage of *incidents* from all the fees payments.
>  
> First we will merge the two dataframes using the `id` from `cash_df` and `cash_request_id` from `fees_df` as indexes.
>  
> We will do a `inner` merge to assure that only the rows where there is a match between `id` and `cash_request_id` in both dataframes are included.

In [ ]:
merged_df = pd.merge(cash_df, fees_df, left_on='id', right_on='cash_request_id', how='inner', suffixes=['_cash', '_fees'])
merged_df

> We can now create a new dataframe with only the relevant columns: `user_id`, `type`, `category` and `cohort`

In [ ]:
incidents_df = merged_df[['user_id', 'type', 'category', 'cohort']]
incidents_df.sample(10)

> We calculate the incident rate by:  
> - getting the count of all payments  
> - counting all the incidents - all the payments where `category` is not null.  
>  
> We group them by `cohort` to create a series so we can see the distribution per cohort.

In [ ]:
total_payments = len(incidents_df)
incidents = incidents_df[incidents_df['category'].notna()].groupby('cohort').size()
incident_rate = (incidents / total_payments) * 100

> As `incidents` now contains a series, we need to convert it to a dataframe to plot it.

In [ ]:
incident_rate_df = incident_rate.to_frame(name='Incident Rate per Cohort').reset_index()
incident_rate_df.columns = ['Cohort', 'Incident Rate (%)']
display(incident_rate_df)

:::{.callout-tip}
We will also export this dataframe to a csv file as we'll need it later.
:::

In [ ]:
pd.DataFrame.to_csv(incident_rate_df, 'incidents.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 6))
incident_plot = sns.barplot(data=incident_rate_df, x='Cohort', y='Incident Rate (%)', palette='magma_r')
plt.title('Incident Rate per Cohort')
plt.ylabel('Incident Rate (%)')
plt.xlabel('Cohort')
incident_plot.bar_label(incident_plot.containers[0], fmt='%.2f%%')
plt.show()

### 3. Revenue Generated by the Cohort
**Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.**

> To calculate the revenue generate per cohort we need to again look at both `cash_df` and `fees_df` dataframes.  
> We will use `merge_df` with the columns `user_id` and `cohort` from `cash_df` and `total_amount` from `fees_df`.  
> We'll also take the `paid_at` column from `fees_df` as we'll consider revenue only payments that have been actually paid - we'll ignore rows that don't have a value in the `paid_at` column.

In [ ]:
revenue_df = merged_df[['user_id', 'cohort', 'total_amount', 'paid_at']]
revenue_df.sample(5)

> Now we can filter out the rows that don't have a value for `paid_at`

In [ ]:
revenue_df = revenue_df[revenue_df['paid_at'].notna()]

> And now we can calculate the revenue generated by each cohort.

In [ ]:
cohort_revenue = revenue_df.groupby('cohort')['total_amount'].sum()
cohort_revenue

In [ ]:
# We save a csv for later use
pd.DataFrame.to_csv(revenue_df, 'revenue.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 6))
revenue_plot = sns.barplot(x=cohort_revenue.index.astype(str), y=cohort_revenue.values, palette='magma_r')
plt.title('Revenue Generated per Cohort')
plt.xlabel('Cohort')
plt.ylabel('Revenue')
# plt.xticks(rotation=45)
plt.tight_layout()
revenue_plot.bar_label(revenue_plot.containers[0], fmt='{:,.0f}')
plt.show()

### 4. New Relevant Metric 
**Propose and calculate a new relevant metric that provides additional insights into user behavior or the performance of IronHack Payments' services.**

> Given the data we have there are a couple of relevant metrics we could investigate that would provide additional insights into user behaviour.  
> One such metric could be **churn rate** - the percentage of users who stop using the IronHack Payments.  
>
> To calculate the **churn_rate** we will use the `cash_df` dataframe and calculate the percentage of deleted user accounts per cohort.

In [ ]:
deleted_accounts = cash_df.groupby('cohort')['deleted_account_id'].count()
deleted_df = deleted_accounts
deleted_df

In [ ]:
# we already have a dataframe with created account per cohort
churn_df = pd.merge(left=deleted_df, right=cohort_counts.reset_index(), left_on='cohort', right_on='cohort')
churn_df.columns = [['cohort', 'deleted_accounts', 'new_accounts']]

:::{.callout-tip}
To calculate the **churn rate** we use the formula:  
$$
{Churn Rate} = \frac{Number of Deleted Accounts}{Number of New Accounts + Number of Deleted Accounts} \times 100
$$
:::
 

In [ ]:
# we define a function that calculates the churn rate for a row
def calculate_churn_rate(row):
    if row['new_accounts'] == 0:  # To avoid division by zero
        return 0
    else:
        return float(row['deleted_accounts'] / (row['new_accounts'] + row['deleted_accounts']) * 100)

# and we call `.apply` on the `churn_df`; this will return a series
churn_rates = churn_df.apply(calculate_churn_rate, axis=1)

# and we assign the series to a new column
churn_df['churn_rate'] = churn_rates

churn_df

In [ ]:
plt.figure(figsize=(10, 6))
churn_plot = sns.barplot(x=churn_df['cohort'].unstack(0), y=churn_rates.values, data=churn_df, palette='magma_r')
plt.title('Churn Rate per Cohort')
plt.xlabel('Cohort')
plt.ylabel('Churn Rate (%)')
# plt.xticks(rotation=45)
churn_plot.bar_label(churn_plot.containers[0], fmt='%.1f%%')
plt.show()

## Final considerations

The exported CSV files will be used for importing in Tableau to generate a dashboard.